# Equilibrium

Equilibrium involves 2 systems of equations:
- Stoichiometry
- Mass Conservation

These two equations operate in different scales. The equilibrium is most easily expressed in log scale, but the mass conservation. 

For this example system, we have the following equilibrium equations:

\begin{aligned}
CO_{2(aq)} + H_2O &\leftrightarrow H^+ + HCO_3^{-} \\
CO_3^{2-} + H^+ &\leftrightarrow HCO_3^- \\
H_2O &\leftrightarrow H^+ + OH^- \\
\end{aligned}

So, excluding water, then we have the following species:

**Primary**:
- $H^+$ 
- $HCO_3^-$
 
**Secondary**:
- $CO_{2(aq)}$
- $CO_3^{2-}$
- $OH^-$

The equations can then be expressed in terms of matrix/vector equations:

\begin{equation*}
\begin{bmatrix}
CO_{2(aq)} \\
CO_3^{2-} \\
OH^- 
\end{bmatrix}
= \begin{bmatrix}
-1 & -1 \\
1 & -1 \\
1 & 0 \\
\end{bmatrix} \begin{bmatrix} H^+ \\ HCO_3^- \end{bmatrix}
\end{equation*}

In [1]:
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import fsolve
from scipy import linalg as la

In [50]:
prim_species: list[str] = ["H+", "HCO3-"]
sec_species: list[str] = ["CO2(aq)", "CO3--", "OH-"]
num_primary: int = len(prim_species)
num_secondary: int = len(sec_species)
small_stoich_mat: NDArray = np.array([
    # [H+, HCO3-]
    [1, 1], # CO2(aq)
    [-1, 1], # CO3--
    [-1, 0] # OH-
], dtype=float)
stoich_mat: NDArray = np.concat([small_stoich_mat, -np.eye(num_secondary)], axis=1)

log_k_w: NDArray = np.array([-6.35, 10.33, 14], dtype=float)
null: NDArray = la.null_space(stoich_mat)
x_p: NDArray 
x_p, resid, rank, sv = la.lstsq(stoich_mat, log_k_w) # type: ignore

In [51]:
def conc(x: NDArray) -> NDArray:
    """
    Takes a vector of length `num_primary`
    """
    return 10 ** (null @ x + x_p)

def check_err(x: NDArray) -> NDArray:
    log_conc: NDArray = np.log(conc(x))
    log_iap: NDArray = stoich_mat @ log_conc
    err = log_iap - log_k_w
    return err

In [52]:
conc(np.array([-1, -1]))

array([9.76206773e-08, 1.07712037e+01, 2.35399781e+00, 5.16086409e-03,
       1.02437314e-07])

In [58]:
def print_stoichiometry(stoich_mat: NDArray, species: list[str]) -> None:
    row_tokens: list[tuple[list[str], list[str]]] = []
    for row in stoich_mat:
        left_tokens: list[str] = []
        right_tokens: list[str] = []
        for nu_i, sp_i in zip(row, species):
            if abs(nu_i) < 1e-3:
                if nu_i > 0:
                    left_tokens.append(f"{abs(nu_i)}{sp_i}")
                else:
                    right_tokens.append(f"{abs(nu_i)} {sp_i}")
        
        row_tokens.append((left_tokens, right_tokens))

    # Construct each side of the equation
    equations: list[str] = []
    for row in row_tokens:
        left: str = " + ".join(row[0])
        right: str = " + ".join(row[1])
        equations.append(left + " = " + right)

    # Print the equations
    for eq in equations:
        print(eq)

In [59]:
print_stoichiometry(stoich_mat, prim_species + sec_species)

 = 0.0 CO3-- + 0.0 OH-
 = 0.0 CO2(aq) + 0.0 OH-
 = 0.0 HCO3- + 0.0 CO2(aq) + 0.0 CO3--


## Total concentrations
The total concentrations involve both the total amount of $HCO_3^-$ in any form, be it $HCO_3^-$, $CO_{2(aq)}$, or $CO_3^{2-}$. The second equation is the charge balance equation, which is always equal to 0. 

In [61]:
c_tot: NDArray = np.array([0.0, 1.0]) # Mass and charge balance
charges: NDArray = np.array([1, -1, 0, -1, -1], dtype=float)
tot_mat: NDArray = np.concat([np.eye(num_primary), abs(small_stoich_mat)], axis=0).T
tot_mat[0] = charges
tot_mat

array([[ 1., -1.,  0., -1., -1.],
       [ 0.,  1.,  1.,  1.,  0.]])

In [62]:
def func(x: NDArray) -> NDArray:
    return c_tot - tot_mat @ conc(x)

In [63]:
x_0: NDArray = np.zeros_like(c_tot)
sol = fsolve(func, x_0)
final_conc = conc(sol)
for c_i, n_i in zip(final_conc, prim_species + sec_species):
    print(f"{n_i}: {c_i:.2e}")

H+: 6.68e-04
HCO3-: 6.68e-04
CO2(aq): 9.99e-01
CO3--: 4.68e-11
OH-: 1.50e-11


In [64]:
tot_mat @ final_conc

array([3.52021916e-15, 1.00000000e+00])